In [44]:
import pandas as pd
import plotly.express as px
import numpy as np
import streamlit as st
import altair as alt

pd.set_option('display.max_columns', None)

In [56]:
usecols = ['market','player_name','datagolf_base_history_fit','draftkings','fanduel','betmgm']

In [58]:
# get american odds

win_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=win&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
win = round(pd.read_csv(win_path, usecols=usecols),2)

top5_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_5&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top5 = round(pd.read_csv(top5_path, usecols=usecols),2)

top10_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_10&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top10 = round(pd.read_csv(top10_path, usecols=usecols),2)

top20_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_20&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top20 = round(pd.read_csv(top20_path, usecols=usecols),2)

# mc_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=make_cut&odds_format=american&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
# make_cut = round(pd.read_csv(mc_path, usecols=usecols),2)

american_odds = pd.concat([win,top5,top10,top20])
american_odds = american_odds[['market','player_name','datagolf_base_history_fit','fanduel','draftkings','betmgm']].convert_dtypes()

In [66]:
# get decimal odds - make expected values

win_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=win&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
win = round(pd.read_csv(win_path, usecols=usecols),3)

top5_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_5&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top5 = round(pd.read_csv(top5_path, usecols=usecols),3)

top10_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_10&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top10 = round(pd.read_csv(top10_path, usecols=usecols),3)

top20_path = r"https://feeds.datagolf.com/betting-tools/outrights?tour=pga&market=top_20&odds_format=percent&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
top20 = round(pd.read_csv(top20_path, usecols=usecols),3)

dec_odds = pd.concat([win,top5,top10,top20])
dec_odds = dec_odds[['market','player_name','datagolf_base_history_fit','fanduel','draftkings','betmgm']]

dec_odds['fanduel_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['fanduel']) - 1,2)
dec_odds['draftkings_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['draftkings']) - 1,2)
dec_odds['betmgm_ev'] = round(dec_odds['datagolf_base_history_fit'] * (1/dec_odds['betmgm']) - 1,2)

ev = dec_odds[['market','player_name','fanduel_ev','draftkings_ev','betmgm_ev']]

In [68]:
# merge

odds = pd.merge(american_odds,ev,how='left',on=['market','player_name']).round(2)
odds = odds[['market','player_name','datagolf_base_history_fit','fanduel','fanduel_ev','draftkings','draftkings_ev','betmgm','betmgm_ev']]
odds

,market,player_name,datagolf_base_history_fit,fanduel,fanduel_ev,draftkings,draftkings_ev,betmgm,betmgm_ev
0,win,"Scheffler, Scottie",584,450,-0.20,450,-0.20,400,-0.27
1,win,"McIlroy, Rory",1348,1100,-0.17,1100,-0.17,1100,-0.17
2,win,"Schauffele, Xander",1362,1400,0.01,1400,0.01,1400,0.01
3,win,"Rahm, Jon",1600,1100,-0.29,1100,-0.29,1200,-0.23
4,win,"Matsuyama, Hideki",2530,2000,-0.21,2000,-0.21,1800,-0.28
...,...,...,...,...,...,...,...,...,...
351,top_20,"Weir, Mike",27219,3500,-0.86,3500,-0.86,3500,-0.86
352,top_20,"Olazabal, Jose Maria",890622,20000,-1.00,20000,-1.00,5000,-1.00
353,top_20,"Singh, Vijay",24295,5500,-0.78,5500,-0.78,2800,-0.88
354,top_20,"Stubbs, Jasper",559256,11000,-1.00,11000,-1.00,2000,-1.00


In [ ]:
pd.melt(odds_master,id_vars=['player_name','market'],value_vars=['datagolf_baseline', 'fanduel', 'fanduel_val',
       'draftkings', 'draftkings_val', 'betmgm', 'betmgm_val', 'caesars',
       'caesars_val'])

In [11]:
odds_master = odds[['market','player_name','datagolf_base_history_fit','fanduel','fanduel_val','draftkings','draftkings_val','betmgm','betmgm_val']].dropna()#,'caesars','caesars_val']]
odds_master.columns = ['Market','Player','Odds','FD Odds','FD EV','DK Odds','DK EV','BetMGM Odds','MGM EV']#,'Caesars','Caesars EV']


odds_dict = {
    'win':'Win',
    'top_5':'Top 5',
    'top_10':'Top 10',
    'top_20':'Top 20'
}
odds_master['Market'] = odds_master['Market'].map(odds_dict)

,Market,Player,Odds,FD Odds,FD EV,DK Odds,DK EV,BetMGM Odds,MGM EV
0,Win,"Scheffler, Scottie",584.0,450.0,0.30,450,0.30,400,0.41
1,Win,"McIlroy, Rory",1348.0,1100.0,0.23,1100,0.23,1100,0.23
2,Win,"Schauffele, Xander",1362.0,1600.0,-0.15,1600,-0.15,1400,-0.02
3,Win,"Rahm, Jon",1600.0,1100.0,0.45,1100,0.45,1200,0.36
4,Win,"Matsuyama, Hideki",2530.0,2000.0,0.26,2000,0.26,1800,0.36
...,...,...,...,...,...,...,...,...,...
84,Top 20,"Weir, Mike",27219.0,4000.0,5.80,4000,5.80,3500,5.93
85,Top 20,"Olazabal, Jose Maria",890622.0,20000.0,43.53,20000,43.53,5000,44.28
86,Top 20,"Singh, Vijay",24295.0,5500.0,3.42,5500,3.42,2800,3.91
87,Top 20,"Stubbs, Jasper",559256.0,11000.0,49.84,11000,49.84,2000,50.66
